In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import sys
import function as func

In [2]:
# Constants
v_c = 220.e+3 # [m/sec] speed of solar system
v_E = v_c # [m/sec] speed of earth
c = 299792458. # [m/sec] speed of light from wikipedia
k_B = 1.380649e-23 # [J/K] boltzmann constant
rbw = 3.e+2 # [Hz]
binwidth = 2.e+3 # [Hz]
TLN2 = 77 # [K]

In [7]:
if __name__ == "__main__":
    """
    check_freq = np.array([18190, 18336, 19120, 19186, 19440, 19478, 19766, 19794, 19818, 20006, 20296, 20302, 20490, 20540, 20892, 21442, 21808, 22522, 22672, 23306, 23808, 23934, 25328, 25352, 25860, 26274, 26346])
    for p in check_freq:
        start_freq = p/10
        #print(start_freq)
        
        initial = int(start_freq * 1.e+6 - 250)
        final = int(initial + 1.e+5)
    """

    yfactor = func.csv_to_array("/data/ms2840a/result_data/check_result/yfactor_check_result.csv")

    check_freq = np.array([18190, 18336, 19120, 19186, 19440, 19478, 19766, 19794, 19818, 20006, 20296, 20302, 20490, 20540, 20892, 21442, 21808, 22522, 22672, 23306, 23808, 23934, 25328, 25352, 25860, 26274, 26346])

    for i in check_freq:
        word = list(str(i*1000 - 250))
        word.insert(2, ".")
        start = "".join(word)
        print(start)

        Gain = 0
        Trx = 0
        Gain_err = 0
        Trx_err = 0
        for _f, freq in enumerate(yfactor["freq"]):
            if float(freq) * 1e+6 == float(start) * 1e+6:
                Gain = yfactor["Gain"][_f]
                Trx = yfactor["Trx"][_f]
                Gain_err = yfactor["Gain_err"][_f]
                Trx_err = yfactor["Trx_err"][_f]
                break

        W_twelve = []
        freq = np.array([])
        for j in range(12):
            path = "/data/ms2840a/signal_data_check/2022-01-17/data/scan_FFT_" + start + "GHz_span2.50MHz_rbw0.3kHz_20.0sec_1counts_12runs_" + str(j) + ".dat"
            freq, W = func.dat_to_array(path)
            W_twelve.append(W/Gain - k_B*Trx*rbw)

        column = ["freq"] + ["W_{}".format(str(x)) for x in range(12)]
        with open("/data/ms2840a/result_data/check_result/signal_12_data/start_{}GHz.csv".format(start), "w") as f:
            writer = csv.writer(f)
            writer.writerow(column)
            for j in range(len(freq)):
                writer.writerow([freq[j]] + [W_twelve[x][j] for x in range(12)])

18.189750
18.335750
19.119750
19.185750
19.439750
19.477750
19.765750
19.793750
19.817750
20.005750
20.295750
20.301750
20.489750
20.539750
20.891750
21.441750
21.807750
22.521750
22.671750
23.305750
23.807750
23.933750
25.327750
25.351750
25.859750
26.273750
26.345750
